In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from torchmetrics import Accuracy, Precision, Recall, F1Score, JaccardIndex
from torchmetrics.collections import MetricCollection
import torchvision.transforms.functional as TF
import random
import satlaspretrain_models
import rasterio
import glob
import re
import traceback

BASE_DATA_DIR_ACTUAL = "PLACEHOLDER_PATH_TO_DATA"
CHECKPOINT_SAVE_DIR_ACTUAL = "PLACEHOLDER_PATH_TO_CHECKPOINTS"
LOGS_DIR_ACTUAL = "PLACEHOLDER_PATH_TO_LOGS"

PATCHES_DIR = os.path.join(BASE_DATA_DIR_ACTUAL, "GEE_WeeklyPatches")
TARGETS_DIR = os.path.join(BASE_DATA_DIR_ACTUAL, "GEE_WeeklyMasks")

os.makedirs(CHECKPOINT_SAVE_DIR_ACTUAL, exist_ok=True)
os.makedirs(LOGS_DIR_ACTUAL, exist_ok=True)

print(f"Data Dir: {BASE_DATA_DIR_ACTUAL}")
print(f"Checkpoints Dir: {CHECKPOINT_SAVE_DIR_ACTUAL}")
print(f"Logs Dir: {LOGS_DIR_ACTUAL}")
print(f"INFO: allenai/satlaspretrain_models will use its default cache path (usually ~/.cache/torch/hub/satlaspretrain/)")

if not os.path.isdir(PATCHES_DIR): print(f"WARNING: Patches directory not found: {PATCHES_DIR}")
if not os.path.isdir(TARGETS_DIR): print(f"WARNING: Masks directory not found: {TARGETS_DIR}")

SATLAS_MODEL_IDENTIFIER = "Sentinel2_SwinB_SI_MS"

RAW_BANDS_IN_GEOTIFF_ORDER = ['B2','B3','B4','B8','B11','B12']
NUM_RAW_BANDS_AVAILABLE = len(RAW_BANDS_IN_GEOTIFF_ORDER)
RAW_BAND_TO_IDX_MAP = {band_name: i for i, band_name in enumerate(RAW_BANDS_IN_GEOTIFF_ORDER)}

SATLAS_INPUT_CHANNEL_NAMES = [
    "TCI_R", "TCI_G", "TCI_B",
    "B05_duplicate", "B06_duplicate", "B07_duplicate",
    "B08_from_B8", "B11", "B12"
]
NUM_SATLAS_INPUT_BANDS = len(SATLAS_INPUT_CHANNEL_NAMES)

SATLAS_CHANNEL_TO_SOURCE_RAW_BAND_MAP = {
    "TCI_R": "B4", "TCI_G": "B3", "TCI_B": "B2",
    "B05_duplicate": "B4", "B06_duplicate": "B3", "B07_duplicate": "B2",
    "B08_from_B8": "B8", "B11": "B11", "B12": "B12"
}
NORMALIZATION_DIVISOR = 8160.0

NUM_CLASSES = 2
IMG_SIZE = 256

BATCH_SIZE = 2; EPOCHS = 100; LR = 1e-5; WEIGHT_DECAY = 0.01
FREEZE_BACKBONE_FPN = False

EARLY_STOPPING_PATIENCE = 15; EARLY_STOPPING_MONITOR = "val_loss"

USE_CUDA_IF_AVAILABLE = True
if USE_CUDA_IF_AVAILABLE and torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"
print(f"Device selected: {DEVICE} (CUDA available: {torch.cuda.is_available()})")

for name in SATLAS_INPUT_CHANNEL_NAMES:
    source = SATLAS_CHANNEL_TO_SOURCE_RAW_BAND_MAP.get(name)
    if not source or source not in RAW_BAND_TO_IDX_MAP:
        print(f"CRITICAL WARNING: Source band for Satlas channel '{name}' ('{source}') is not valid. Check config.")
if NUM_SATLAS_INPUT_BANDS != 9: print(f"CRITICAL WARNING: NUM_SATLAS_INPUT_BANDS is {NUM_SATLAS_INPUT_BANDS}, Satlas MS expects 9.")

class CerealSensingDataset(Dataset):
    def __init__(self, file_identifiers, patches_dir, targets_dir, target_img_size,
                 raw_band_to_idx_map, satlas_input_channel_names,
                 satlas_channel_to_source_raw_band_map, normalization_divisor
                ):

        self.file_identifiers = file_identifiers; self.patches_dir = patches_dir; self.targets_dir = targets_dir
        self.target_img_size = target_img_size; self.raw_band_to_idx_map = raw_band_to_idx_map
        self.satlas_input_channel_names = satlas_input_channel_names
        self.satlas_channel_to_source_raw_band_map = satlas_channel_to_source_raw_band_map
        self.normalization_divisor = normalization_divisor

    def __len__(self): return len(self.file_identifiers)

    def _load_process_and_normalize_image(self, image_identifier):
        image_filename = f"{image_identifier}_S2.tif"
        image_path = os.path.join(self.patches_dir, image_filename)
        with rasterio.open(image_path) as src: raw_image_data_from_file = src.read()
        processed_bands_for_satlas = []
        for target_satlas_channel_name in self.satlas_input_channel_names:
            source_raw_band_name = self.satlas_channel_to_source_raw_band_map[target_satlas_channel_name]
            raw_band_idx_in_file = self.raw_band_to_idx_map[source_raw_band_name]
            processed_bands_for_satlas.append(raw_image_data_from_file[raw_band_idx_in_file])
        satlas_stacked_image_np = np.stack(processed_bands_for_satlas, axis=0).astype(np.float32)
        image_tensor = torch.from_numpy(satlas_stacked_image_np)
        if image_tensor.shape[1:] != (self.target_img_size, self.target_img_size):
            image_tensor = F.interpolate(image_tensor.unsqueeze(0),
                                         size=(self.target_img_size, self.target_img_size),
                                         mode='bilinear', align_corners=False).squeeze(0)
        image_tensor = image_tensor / self.normalization_divisor
        image_tensor = torch.clip(image_tensor, 0.0, 1.0)
        return image_tensor

    def __getitem__(self, idx):
        identifier = self.file_identifiers[idx]
        try:
            image_tensor = self._load_process_and_normalize_image(identifier)
            mask_filename = f"{identifier}_Mask.tif"; label_path = os.path.join(self.targets_dir, mask_filename)
            with rasterio.open(label_path) as src: label = src.read(1)
            label_tensor = F.interpolate(torch.tensor(label, dtype=torch.float32).unsqueeze(0).unsqueeze(0),
                                         size=(self.target_img_size, self.target_img_size), mode='nearest').squeeze(0).squeeze(0).long()

            if image_tensor.shape!=(NUM_SATLAS_INPUT_BANDS,self.target_img_size,self.target_img_size) or \
               label_tensor.shape!=(self.target_img_size,self.target_img_size):
                raise ValueError(f"Shape mismatch for {identifier}. Img:{image_tensor.shape}, Lbl:{label_tensor.shape}")

            return image_tensor, label_tensor
        except Exception as e:
            print(f"ERROR in __getitem__ for {identifier}: {e}"); traceback.print_exc(); raise e

print("\n=========== Data Loading & Prep (Baseline - No Augmentations) ============")
image_files_pattern = os.path.join(PATCHES_DIR, "Patch*_S2.tif")
all_image_filepaths = sorted(glob.glob(image_files_pattern))
initial_file_identifiers = []
if not all_image_filepaths: print(f"ERROR: No image files found in {PATCHES_DIR}")
else: print(f"Found {len(all_image_filepaths)} potential image files. Checking masks...")
for img_path in all_image_filepaths:
    base_name = os.path.basename(img_path); identifier_match = re.match(r"(Patch\d+_W\d+_\d{8})_S2\.tif", base_name)
    if identifier_match:
        identifier = identifier_match.group(1); mask_path = os.path.join(TARGETS_DIR, f"{identifier}_Mask.tif")
        if os.path.exists(mask_path): initial_file_identifiers.append(identifier)
        else: print(f"WARNING: Mask for {identifier} not found. Skipping.")
    else: print(f"WARNING: Could not parse {base_name}. Skipping.")
print(f"Found {len(initial_file_identifiers)} initial image-mask pairs.")

print(f"\nPerforming data integrity check on {len(initial_file_identifiers)} pairs...")
corrupted_identifiers, valid_identifiers = [], []
if not initial_file_identifiers: print("No identifiers to check.")
else:
    for identifier in initial_file_identifiers:
        image_path = os.path.join(PATCHES_DIR, f"{identifier}_S2.tif"); problem, msgs = False, []
        try:
            with rasterio.open(image_path) as src: image_data_raw_check = src.read()
            if image_data_raw_check.shape[0] < NUM_RAW_BANDS_AVAILABLE:
                msgs.append(f"File has {image_data_raw_check.shape[0]} bands, expected {NUM_RAW_BANDS_AVAILABLE}."); problem = True
            else:
                data_to_check_integrity = image_data_raw_check[:NUM_RAW_BANDS_AVAILABLE, :, :].astype(np.float32)
                if np.isnan(data_to_check_integrity).any(): msgs.append("NaNs"); problem = True
                if data_to_check_integrity.min() < -1000: msgs.append(f"Low raw values: {data_to_check_integrity.min()}"); problem = True
                if data_to_check_integrity.max() > 20000: msgs.append(f"High raw values: {data_to_check_integrity.max()}"); problem = True
                if data_to_check_integrity.size > 0 and not problem:
                    is_uniform = True
                    for i in range(data_to_check_integrity.shape[0]):
                        if data_to_check_integrity.shape[1]>0 and data_to_check_integrity.shape[2]>0:
                            if not np.allclose(data_to_check_integrity[i], data_to_check_integrity[i,0,0], atol=1e-1): is_uniform = False; break
                        elif data_to_check_integrity.shape[1] == 0 or data_to_check_integrity.shape[2] == 0: is_uniform = False; break
                    if is_uniform and data_to_check_integrity.shape[1]>0 : msgs.append("All raw bands appear uniform"); problem = True
                elif data_to_check_integrity.size == 0: msgs.append("Empty raw data"); problem = True
            if problem: print(f"WARNING: {identifier} issues: {'; '.join(msgs)}. Skipping."); corrupted_identifiers.append(identifier)
            else: valid_identifiers.append(identifier)
        except Exception as e: print(f"ERROR checking {identifier}: {e}. Skipping."); corrupted_identifiers.append(identifier)
print(f"Integrity check complete. Corrupted: {len(corrupted_identifiers)}.")
clean_identifiers = valid_identifiers; num_clean_patches = len(clean_identifiers)
print(f"Clean patches: {num_clean_patches}")

train_ds, val_ds, train_loader, val_loader = None, None, None, None
if num_clean_patches > 0:
    random.shuffle(clean_identifiers)
    split_idx = int(num_clean_patches * 0.8)
    train_ids, val_ids = (clean_identifiers[:split_idx], clean_identifiers[split_idx:]) if num_clean_patches >=2 else (clean_identifiers, [])
    if not val_ids and len(train_ids) > 1 : val_ids = [train_ids.pop()]
    print(f"Training IDs: {len(train_ids)}, Validation IDs: {len(val_ids)}")

    if train_ids:
        train_ds = CerealSensingDataset(
            file_identifiers=train_ids,
            patches_dir=PATCHES_DIR,
            targets_dir=TARGETS_DIR,
            target_img_size=IMG_SIZE,
            raw_band_to_idx_map=RAW_BAND_TO_IDX_MAP,
            satlas_input_channel_names=SATLAS_INPUT_CHANNEL_NAMES,
            satlas_channel_to_source_raw_band_map=SATLAS_CHANNEL_TO_SOURCE_RAW_BAND_MAP,
            normalization_divisor=NORMALIZATION_DIVISOR
        )
        train_loader = DataLoader(train_ds, BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=False)
        print(f"Train DataLoader: {len(train_loader.dataset)} samples. Augmentations: False")

    if val_ids:
        val_ds = CerealSensingDataset(
            file_identifiers=val_ids,
            patches_dir=PATCHES_DIR,
            targets_dir=TARGETS_DIR,
            target_img_size=IMG_SIZE,
            raw_band_to_idx_map=RAW_BAND_TO_IDX_MAP,
            satlas_input_channel_names=SATLAS_INPUT_CHANNEL_NAMES,
            satlas_channel_to_source_raw_band_map=SATLAS_CHANNEL_TO_SOURCE_RAW_BAND_MAP,
            normalization_divisor=NORMALIZATION_DIVISOR
        )
        val_loader = DataLoader(val_ds, BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=False)
        print(f"Val DataLoader: {len(val_loader.dataset)} samples. Augmentations: False")
    elif not val_ids and train_ids:
        print("WARNING: Validation dataset is empty (but training data exists).")
    elif not train_ids :
        print("WARNING: Training dataset is empty, so validation dataset is also empty.")
else:
    print("ERROR: No clean patches to create datasets.")


if train_ds and len(train_ds) > 0:
    try:
        print("\n--- Visualizing a sample from TRAIN_DS (Baseline - No Augmentations) ---")
        sample_idx = 0 ; sample_img_tensor, sample_mask_tensor = train_ds[sample_idx]
        print(f"Sample Img Shape: {sample_img_tensor.shape}, Mask Shape: {sample_mask_tensor.shape}, Unique Mask: {torch.unique(sample_mask_tensor)}")

        tci_view_tensor = sample_img_tensor[:3, :, :].cpu()
        print(f"TCI View (Ch 0,1,2 - SR data / {NORMALIZATION_DIVISOR}) - Min: {tci_view_tensor.min():.4f}, Max: {tci_view_tensor.max():.4f}, Mean: {tci_view_tensor.mean():.4f}")

        GAIN_FOR_VISUALIZATION = 3.5
        img_for_vis_gained = tci_view_tensor.numpy().transpose(1, 2, 0) * GAIN_FOR_VISUALIZATION
        img_display_np = np.clip(img_for_vis_gained, 0, 1)

        fig, axs = plt.subplots(1,2, figsize=(10,5))
        axs[0].imshow(img_display_np); axs[0].set_title(f'Input ID: {train_ds.file_identifiers[sample_idx]} (TCI, Vis Gain x{GAIN_FOR_VISUALIZATION})\nOriginal Sample'); axs[0].axis('off')
        axs[1].imshow(sample_mask_tensor.cpu().numpy(), cmap='gray'); axs[1].set_title('Mask (Original)'); axs[1].axis('off')
        plt.tight_layout(); plt.show()

        print("\n--- Ranges for ALL 9 Normalized Channels in Sample (all divided by 8160): ---")
        for i, ch_name_vis in enumerate(SATLAS_INPUT_CHANNEL_NAMES):
            ch_data = sample_img_tensor[i, :, :].cpu()
            source_raw_band = SATLAS_CHANNEL_TO_SOURCE_RAW_BAND_MAP[ch_name_vis]
            print(f"  Channel {i} ('{ch_name_vis}' from '{source_raw_band}'): Min={ch_data.min():.6f}, Max={ch_data.max():.6f}, Mean={ch_data.mean():.6f}")

    except Exception as e: print(f"Visualization error: {e}"); traceback.print_exc()
else: print("Training dataset empty. Skipping visualization.")

print(f"\n--- Loading Satlas Model via weights_manager: {SATLAS_MODEL_IDENTIFIER} ---")
satlas_full_segmentation_model = None
try:
    weights_manager = satlaspretrain_models.Weights()

    satlas_full_segmentation_model = weights_manager.get_pretrained_model(
        model_identifier=SATLAS_MODEL_IDENTIFIER,
        fpn=True,
        head=satlaspretrain_models.Head.SEGMENT,
        num_categories=NUM_CLASSES
    )
    print(f"Successfully loaded Satlas model with segmentation head: {SATLAS_MODEL_IDENTIFIER}")

except Exception as e:
    print(f"Error loading Satlas model via weights_manager: {e}")
    traceback.print_exc()
    satlas_full_segmentation_model = None

class LitSegmentationModel(LightningModule):
    def __init__(self, full_satlas_model, learning_rate, weight_decay, num_classes, class_names=None, freeze_backbone_fpn=False):
        super().__init__()
        self.save_hyperparameters(ignore=['full_satlas_model'])
        self.model = full_satlas_model

        if freeze_backbone_fpn:
            frozen_count = 0
            if hasattr(self.model, 'backbone'):
                for param in self.model.backbone.parameters(): param.requires_grad = False
                print("Satlas backbone parameters frozen.")
                frozen_count +=1
            if hasattr(self.model, 'fpn'):
                 for param in self.model.fpn.parameters(): param.requires_grad = False
                 print("Satlas FPN parameters frozen.")
                 frozen_count +=1
            if frozen_count == 0: print("WARNING: Could not find 'backbone' or 'fpn' attributes directly on self.model to freeze.")
            if hasattr(self.model, 'head'): print("Segmentation head (self.model.head) parameters are expected to be trainable.")

        self.class_weights_tensor = None
        self.class_names = class_names if class_names else [f"class_{i}" for i in range(num_classes)]

        metrics = MetricCollection({
            'accuracy_overall': Accuracy(task="multiclass", num_classes=num_classes, average='micro'),
            'f1_score_macro': F1Score(task="multiclass", num_classes=num_classes, average='macro'),
            'jaccard_index_macro': JaccardIndex(task="multiclass", num_classes=num_classes, average='macro'),
            'accuracy_per_class': Accuracy(task="multiclass", num_classes=num_classes, average='none'),
            'jaccard_index_per_class': JaccardIndex(task="multiclass", num_classes=num_classes, average='none'),
            'f1_score_per_class': F1Score(task="multiclass", num_classes=num_classes, average='none')
        })
        self.val_metrics = metrics.clone(prefix='val_')

    def forward(self, x):
        model_output = self.model(x)
        if isinstance(model_output, tuple):
            logits = model_output[0]
        else:
            logits = model_output
        return logits

    def training_step(self, batch, batch_idx):
        x, y = batch; logits = self.forward(x)
        loss = F.cross_entropy(logits, y, weight=(self.class_weights_tensor.to(logits.device) if self.class_weights_tensor is not None else None))
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True, sync_dist=True); return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch; logits = self.forward(x)
        loss = F.cross_entropy(logits, y, weight=(self.class_weights_tensor.to(logits.device) if self.class_weights_tensor is not None else None))
        self.log("val_loss", loss, prog_bar=True, logger=True, sync_dist=True); self.val_metrics.update(torch.argmax(logits, dim=1), y); return loss

    def on_validation_epoch_end(self):
        metrics_output = self.val_metrics.compute()
        for name, value in metrics_output.items():
            is_prog_bar = name in ["val_f1_score_macro", "val_jaccard_index_macro"]
            if isinstance(value, torch.Tensor) and value.numel() == self.hparams.num_classes:
                for i in range(self.hparams.num_classes):
                    class_metric_name = f"{name}_{self.class_names[i]}"
                    prog_bar_metric_name, log_to_prog_bar = None, False
                    if self.class_names[i] == "cereal":
                        if name == "val_jaccard_index_per_class":
                            prog_bar_metric_name, log_to_prog_bar = "val_iou_cereal", True
                        elif name == "val_f1_score_per_class":
                            prog_bar_metric_name, log_to_prog_bar = "val_f1_cereal", True
                    if prog_bar_metric_name and log_to_prog_bar:
                        self.log(prog_bar_metric_name, value[i], prog_bar=True, logger=False, sync_dist=True)
                        self.log(class_metric_name, value[i], prog_bar=False, logger=True, sync_dist=True)
                    else:
                        self.log(class_metric_name, value[i], prog_bar=False, logger=True, sync_dist=True)
            else:
                self.log(name, value, prog_bar=is_prog_bar, logger=True, sync_dist=True)
        self.val_metrics.reset()

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.hparams.learning_rate, weight_decay=self.hparams.weight_decay)

CLASS_NAMES_FOR_LOGGING = ["non_cereal", "cereal"]
lightning_model = None
if 'satlas_full_segmentation_model' in locals() and satlas_full_segmentation_model is not None:
    lightning_model = LitSegmentationModel(
        full_satlas_model=satlas_full_segmentation_model,
        learning_rate=LR,
        weight_decay=WEIGHT_DECAY,
        num_classes=NUM_CLASSES,
        class_names=CLASS_NAMES_FOR_LOGGING,
        freeze_backbone_fpn=FREEZE_BACKBONE_FPN
    )
    print("LitSegmentationModel re-instantiated (for baseline - no augmentations).")
else:
    print("ERROR: satlas_full_segmentation_model not loaded (from Cell 6). Cannot instantiate LitSegmentationModel.")

if DEVICE == "cuda" and torch.cuda.is_available():
    if torch.cuda.is_initialized():
        print("WARNING FROM DEBUG: CUDA IS ALREADY INITIALIZED before Trainer instantiation!")
    else:
        print("INFO FROM DEBUG: CUDA is NOT yet initialized before Trainer instantiation. This is ideal.")

csv_logger = CSVLogger(LOGS_DIR_ACTUAL, name="cereal_seg_satlas_baseline_no_aug_run")
print(f"Logs: {csv_logger.log_dir}")
loss_cb = ModelCheckpoint(CHECKPOINT_SAVE_DIR_ACTUAL, 'model_baseline_loss-{epoch:02d}-{val_loss:.3f}', monitor='val_loss', mode='min', save_top_k=1, verbose=True)
iou_cb = ModelCheckpoint(CHECKPOINT_SAVE_DIR_ACTUAL, 'model_baseline_iou_cereal-{epoch:02d}-{val_iou_cereal:.3f}', monitor='val_iou_cereal', mode='max', save_top_k=1, verbose=True)
early_stop_cb = EarlyStopping(EARLY_STOPPING_MONITOR, patience=EARLY_STOPPING_PATIENCE, verbose=True, mode=("min" if "loss" in EARLY_STOPPING_MONITOR else "max"))

if lightning_model and train_loader:
    trainer_accelerator = DEVICE
    trainer_devices_setting = 1
    trainer_strategy = "auto"

    if DEVICE == "cuda" and torch.cuda.is_available():
        print(f"INFO: CUDA is available. Configuring for single GPU training on the first GPU.")
        trainer_devices_setting = [0]
    elif DEVICE == "cpu":
        print(f"INFO: Configuring for CPU training.")
        trainer_accelerator = "cpu"
    else:
        print(f"WARNING: DEVICE='{DEVICE}' but CUDA not actually available. Forcing CPU training.")
        trainer_accelerator = "cpu"
        trainer_devices_setting = 1

    trainer = Trainer(accelerator=trainer_accelerator,
                      devices=trainer_devices_setting,
                      strategy=trainer_strategy,
                      max_epochs=EPOCHS,
                      logger=csv_logger,
                      callbacks=[early_stop_cb, loss_cb, iou_cb],
                      gradient_clip_val=0.5)

    print(f"Trainer initialized: accelerator='{trainer_accelerator}', devices={trainer_devices_setting}, strategy='{trainer_strategy}'")
    print(f"Starting training (Baseline - No Augmentations): {SATLAS_MODEL_IDENTIFIER} (all bands / {NORMALIZATION_DIVISOR})...")
    try:
        val_dl_arg = val_loader if val_loader and hasattr(val_loader, 'dataset') and len(val_loader.dataset)>0 else None
        cbs_to_use = trainer.callbacks
        if not val_dl_arg and cbs_to_use:
            print("No val_loader, adjusting callbacks that monitor validation metrics.")
            original_callbacks = list(cbs_to_use); cbs_to_use = []
            for cb_instance in original_callbacks:
                monitor_attr = getattr(cb_instance, 'monitor', None)
                if monitor_attr and monitor_attr.startswith("val_"): print(f"Removing callback: {cb_instance.__class__.__name__} on {monitor_attr}")
                else: cbs_to_use.append(cb_instance)
            trainer.callbacks = cbs_to_use if cbs_to_use else None

        trainer.fit(lightning_model, train_dataloaders=train_loader, val_dataloaders=val_dl_arg)
        print("Training finished.")
        if loss_cb and hasattr(loss_cb, 'best_model_path') and loss_cb.best_model_path: print(f"Best loss model: {loss_cb.best_model_path}")
        if iou_cb and hasattr(iou_cb, 'best_model_path') and iou_cb.best_model_path: print(f"Best IoU Cereal model: {iou_cb.best_model_path}")
    except Exception as e: print(f"Training error: {e}"); traceback.print_exc()
else: print("ERROR: Training prerequisites not met (lightning_model or train_loader is None).")